The purpose of this notebook is to demonstrate the functionality of the `VARReduce` model from sktime. We will compare its performance with the traditional `VAR` model to highlight their similar behavior under default settings. Additionally, we will showcase how `VARReduce` can be leveraged for regularization in scenarios involving large time series datasets, offering enhanced model performance and robustness.

In [1]:
# Suppress all warnings
import warnings

import numpy as np
import pandas as pd
from statsmodels.tsa.api import VAR

from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.var_reduce import VARReduce
from sktime.performance_metrics.forecasting import mean_absolute_percentage_error

warnings.filterwarnings("ignore")

In [2]:
%load_ext autoreload
%autoreload 2

# Data Generation

To perform the comparison between `VAR` and `VARReduce`, we first generate a small synthetic multivariate time series dataset. 

We create three time series, each with 100 observations, and store them in a DataFrame. 

We then split the data into training and testing sets, reserving the last 5 observations for testing.

In [3]:
np.random.seed(42)
n_obs = 100
time_series_1 = np.random.randn(n_obs) + 1000
time_series_2 = np.random.randn(n_obs) + 100
time_series_3 = np.random.randn(n_obs) + 10

df = pd.DataFrame(
    {
        "ts1": time_series_1,
        "ts2": time_series_2,
        "ts3": time_series_3,
    }
)

# Split the data into training and testing sets
df_train, df_test = temporal_train_test_split(df, test_size=5)

# Define the forecasting horizon
fh = ForecastingHorizon(df_test.index, is_relative=False)

# Model Introduction

The `VARReduce` model applies Vector Autoregression (VAR) with optional regularization. 
It operates by converting multivariate time series data into a tabular format suitable for regression models. By default, `VARReduce` uses `scikit-learn`'s `LinearRegression` to perform the fitting and behaves like a traditional VAR model. However, users can specify other `scikit-learn` compatible regressors to introduce regularization, enhancing performance for large datasets or when multicollinearity is present.

Here's how the `VARReduce` model works step-by-step:

1. Data Preparation: The time series data is transformed into a format where each row contains lagged values of the series, turning it into a tabular dataset.
2. Model Fitting: The model uses the specified regressor to fit the lagged values as predictors and the current values as the response variable.
3. Forecasting: The fitted model is used to forecast future values based on the most recent observations.

To demonstrate the intermediate DataFrame generated during the data preparation step, we'll take a peep at the internal `prepare_var_data` method from the `VARReduce` class.

In [4]:
# Define lags
LAGS = 2

# Instantiate and fit the custom VARReduce model
varreduce_model = VARReduce(
    lags=LAGS
)  # no regressor passed, by default, LinearRegressor() will be used
varreduce_model.fit(df_train)

# Fit the VAR model comparison
var_model = VAR(endog=df_train)

# Fit the VAR model using statsmodels for comparison
var_model = VAR(df_train)
results = var_model.fit(LAGS)


We observe that internally, `VARReduce` transforms the provided training data into a tabular format suitable for regression. Specifically:

1. **Predictors (X)**: The predictors consist of lagged values of the time series, with the number of lags specified by the `lags` parameter. In the displayed DataFrame, each column represents a lagged value for one of the time series (e.g., `ts1_lag1`, `ts2_lag2`).

2. **Target Variables (y)**: The target variables are the current, unlagged values of the time series, shown in columns such as `ts1_lag0`, `ts2_lag0`, and `ts3_lag0`.



In [5]:
X = varreduce_model.prepare_var_data(df_train, return_as_ndarray=False)[0]
X.head()

,ts1_lag1,ts2_lag1,ts3_lag1,ts1_lag2,ts2_lag2,ts3_lag2
2,999.861736,99.579355,10.560785,1000.496714,98.584629,10.357787
3,1000.647689,99.657285,11.083051,999.861736,99.579355,10.560785
4,1001.523030,99.197723,11.053802,1000.647689,99.657285,11.083051
5,999.765847,99.838714,8.622331,1001.523030,99.197723,11.053802
6,999.765863,100.404051,9.062175,999.765847,99.838714,8.622331


In [6]:
y = varreduce_model.prepare_var_data(df_train, return_as_ndarray=False)[1]
y.head()

,ts1_lag0,ts2_lag0,ts3_lag0
2,1000.647689,99.657285,11.083051
3,1001.523030,99.197723,11.053802
4,999.765847,99.838714,8.622331
5,999.765863,100.404051,9.062175
6,1001.579213,101.886186,10.515035


# Coefficient Comparison

Once the data is prepared in this tabular format, `VARReduce` trains a regressor on it. By default, `LinearRegression` from scikit-learn is used as the regressor. Under these default conditions, the `VARReduce` model behaves just like a traditional VAR model. This means that the fitted coefficients and the resulting forecasts will be identical to those of a VAR model, as both are essentially performing linear regression on the same lagged data.

This equivalence is demonstrated below:

In [7]:
# Compare coefficients
print("Coefficients from VARReduce model:")
print(varreduce_model.coefficients)
print("\nCoefficients from statsmodels VAR:")
print(results.coefs)

Coefficients from VARReduce model:
[[[-0.04379616 -0.14362626 -0.02868124]
  [-0.02691002 -0.12388731 -0.09702357]
  [-0.02884349 -0.12771343 -0.11331339]]

 [[-0.0488787  -0.0536462  -0.01244942]
  [ 0.11919954 -0.05735081 -0.13221751]
  [ 0.0930699   0.09744015 -0.0391535 ]]]

Coefficients from statsmodels VAR:
[[[-0.04379616 -0.14362626 -0.02868124]
  [-0.02691002 -0.12388731 -0.09702357]
  [-0.02884349 -0.12771343 -0.11331339]]

 [[-0.0488787  -0.0536462  -0.01244942]
  [ 0.11919954 -0.05735081 -0.13221751]
  [ 0.0930699   0.09744015 -0.0391535 ]]]


# Forecast Comparison

Given that the two models have essentially identical fitted parameters, it is unsurprising that their forecasts are also essentially identical:

In [8]:
# Generating forecasts from both models
df_pred_varreduce = varreduce_model.predict(fh=fh)
df_pred_statsmodels = results.forecast(df_train.values[-LAGS:], steps=len(fh))

# Compare predictions using numpy's allclose function
epsilon = 1e-8
predictions_are_close = np.allclose(
    df_pred_varreduce, df_pred_statsmodels, atol=epsilon
)

print(
    "Are the predictions from VARReduce and sktime VAR close? ", predictions_are_close
)

Are the predictions from VARReduce and sktime VAR close?  True


# Real World Data

In [9]:
import statsmodels.api as sm

# Load the Longley dataset using statsmodels
dataset = sm.datasets.macrodata.load_pandas()
df = dataset.data
df["year"] = df["year"].astype(int)
df["quarter"] = df["quarter"].astype(int)

# Create a datetime column
df["date"] = pd.to_datetime(df["year"].astype(str) + "Q" + df["quarter"].astype(str))

# Set the datetime column as the index
df.set_index("date", inplace=True)

# Convert the datetime index to a period index with quarterly frequency
df.index = df.index.to_period("Q")

# Drop the original year and quarter columns if no longer needed
df.drop(columns=["year", "quarter"], inplace=True)

df.head()

,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
date,,,,,,,,,,,,
1959Q1,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1959Q2,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
1959Q3,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
1959Q4,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
1960Q1,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [10]:
# Split the data into training and testing sets
df_train, df_test = temporal_train_test_split(df, test_size=24)
fh = ForecastingHorizon(df_test.index, is_relative=False)

In [11]:
# Define lags
LAGS = 2

# Instantiate and fit the custom VARReduce model
varreduce_model = VARReduce(
    lags=LAGS
)  # no regressor passed, by default, LinearRegressor() will be used
varreduce_model.fit(df_train)

# Fit the VAR model comparison
var_model = VAR(endog=df_train)

# Fit the VAR model using statsmodels for comparison
var_model = VAR(df_train)
results = var_model.fit(LAGS)

# Generating forecasts from both models
y_pred_varreduce = varreduce_model.predict(fh=fh)
y_pred_statsmodels = results.forecast(df_train.values[-LAGS:], steps=len(fh))

# Compare predictions using numpy's allclose function
epsilon = 1e-8
predictions_are_close = np.allclose(y_pred_varreduce, y_pred_statsmodels, atol=epsilon)

print(
    "Are the predictions from VARReduce and sktime VAR close? ", predictions_are_close
)

Are the predictions from VARReduce and sktime VAR close?  True


In [37]:
from sklearn.linear_model import Ridge

# Instantiate and fit the custom VARReduce model
varreduce_model = VARReduce(
    lags=LAGS, regressor=Ridge(alpha=1000)
)  # no regressor passed, by default, LinearRegressor() will be used
varreduce_model.fit(df_train)

# Fit the VAR model using statsmodels for comparison
var_model = VAR(df_train)
results = var_model.fit(LAGS)

In [38]:
# Generating forecasts from both models
df_pred_varreduce = varreduce_model.predict(fh=fh)
df_pred_var = results.forecast(df_train.values[-LAGS:], steps=len(fh))
df_pred_var = pd.DataFrame(df_pred_var, index=df_test.index, columns=df_test.columns)

In [39]:
# Calculate performance metrics for VARReduce
mape_varreduce = mean_absolute_percentage_error(df_test, df_pred_varreduce)
print(f"MAPE for VARReduce model: {mape_varreduce:.2f}")

# Calculate performance metrics for statsmodels VAR
mape_var = mean_absolute_percentage_error(df_test, df_pred_var)
print(f"MAPE for statsmodels VAR model: {mape_var:.2f}")

MAPE for VARReduce model: 0.58
MAPE for statsmodels VAR model: 0.69
